# **ТЗ для вакансии “Data Scientist Intern”**


**1. Найти стоимость тарифа доставки для каждого склада**

---


Для этого считаем json-файл с данными о заказах, затем для каждого заказа будем вычислять отношение суммарной стоимости доставки всех продуктов к количеству проданного товара. Полученные данные обернем в множество, состоящее из пар: наименование склада и стоимость его тарифа доставки.  

In [8]:
import json
import pandas as pd
import numpy as np

from google.colab import drive
from itertools import product

# Cчитаем данные из json-файла.
drive.mount('/content/drive')
with open('/content/drive/My Drive/trial task/trial_task.json') as json_file:
    data = json.load(json_file)

'''with open('trial_task.json') as json_file:
  data = json.load(json_file)'''


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


"with open('trial_task.json') as json_file:\n  data = json.load(json_file)"

In [9]:
''' Для каждого заказа будем вычислять отношение суммарной стоимости
    доставки всех продуктов к количеству проданного товара. '''
delivery_cost_rate_for_each_warehouse = set()

for i in range(len(data)):
    total_quantity_for_each_warehouse = 0
    for product in data[i]['products']:
        total_quantity_for_each_warehouse += product['quantity']

    delivery_cost_rate_for_each_warehouse.add(
                (data[i]['warehouse_name'],
                int(data[i]['highway_cost'] /
                total_quantity_for_each_warehouse)))

In [10]:
# Из полученного множества создадим DataFrame.
delivery_cost_rate_df = pd.DataFrame(
        data = delivery_cost_rate_for_each_warehouse,
        columns = ['warehouse_name', 'delivery_cost_rate'])

display(delivery_cost_rate_df)

,warehouse_name,delivery_cost_rate
0,хутор близ Диканьки,-15
1,Мордор,-10
2,гиперборея,-20
3,отель Лето,-25
4,остров невезения,-5


**2.  Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами
'product', 'quantity', 'income', 'expenses', 'profit')**


---




In [35]:
# Создаём пустой DataFrame с заданными столбцами.
columns = [
    'product',
    'quantity',
    'income',
    'expenses',
    'profit'
]
sum_indicators_for_each_product = pd.DataFrame(columns=columns)

In [13]:
# Определяем, сколько существует уникальных товаров.
products_set = set()

for i in range(len(data)):
   for product in data[i]['products']:
        products_set.add(product['product'])
products_set

{'автограф Стаса Барецкого',
 'билет в Израиль',
 'зеленая пластинка',
 'ломтик июльского неба',
 'плюмбус',
 'подписка на suppi-блог',
 'статуэтка Ленина'}

In [36]:
# Занесём данные c названиями продуктов в таблицу, а остальное инициализируем нулями.
sum_indicators_for_each_product['product'] = list(products_set)

for column in columns[1:]:
    sum_indicators_for_each_product[column] = np.zeros(len(products_set))

display(sum_indicators_for_each_product)

,product,quantity,income,expenses,profit
0,билет в Израиль,0.0,0.0,0.0,0.0
1,зеленая пластинка,0.0,0.0,0.0,0.0
2,автограф Стаса Барецкого,0.0,0.0,0.0,0.0
3,плюмбус,0.0,0.0,0.0,0.0
4,статуэтка Ленина,0.0,0.0,0.0,0.0
5,подписка на suppi-блог,0.0,0.0,0.0,0.0
6,ломтик июльского неба,0.0,0.0,0.0,0.0


Опишем функции для вычисления основных суммарных показателей для продукта

In [17]:
# Вычисление суммарного количества переданного продукта.
def compute_sum_quantity_for_product(product_name):
    quantity = 0
    for i in range(len(data)):
        for product in data[i]['products']:
            if product_name == product['product']:
                quantity += product['quantity']
    return quantity

In [18]:
# Вычисление суммарного дохода с продукта.
def compute_sum_income_for_product(product_name):
    sum_income = 0
    for i in range(len(data)):
        for product in data[i]['products']:
            if product_name == product['product']:
                sum_income += product['price'] * product['quantity']
    return sum_income

In [20]:
# Получения тарифа стоимости доставки для определенного склада.
def get_delivery_cost_rate(warehouse_name):
    index = delivery_cost_rate_df.index[
                    delivery_cost_rate_df['warehouse_name'] == warehouse_name]
    return int(delivery_cost_rate_df.iloc[index]['delivery_cost_rate'])

In [21]:
# Вычисление суммарных расходов на продукт.
def compute_sum_expenses_for_product(product_name):
    sum_expenses = 0
    for i in range(len(data)):
        warehouse_name = data[i]['warehouse_name']
        delivery_cost_rate = get_delivery_cost_rate(warehouse_name)

        for product in data[i]['products']:
            if product_name == product['product']:
                sum_expenses += product['quantity'] * delivery_cost_rate
    return sum_expenses

In [23]:
# Вычисление суммарной прибыли для продукта.
def compute_sum_profit_for_product(product_name):
    income = compute_sum_income_for_product(product_name)
    expenses = compute_sum_expenses_for_product(product_name)
    return income + expenses

In [24]:
# Заполнение суммарного показателя для каждого продукта.
def fill_in_the_product(compute):
 # В качестве аргумента передаем функцию,
 # вычисляющую определенный суммарный показатель.
    product_indicators = []
    for index, product_name in enumerate(
                                sum_indicators_for_each_product['product']):
        product_indicators.append(compute(product_name))
    return product_indicators

Заполним столбцы таблицы sum_indicators_for_each_product


In [38]:
# Заполняем столбцы таблицы sum_indicators_for_each_product.
# Cоздадим массиво из функций, вычисляющих суммарные показатели
functions = [
        compute_sum_quantity_for_product,
        compute_sum_income_for_product,
        compute_sum_expenses_for_product,
        compute_sum_profit_for_product]

for function, column in zip(functions, columns[1:]):
    sum_indicators_for_each_product[column] = fill_in_the_product(function)

display(sum_indicators_for_each_product)

,product,quantity,income,expenses,profit
0,билет в Израиль,58,58000,-1050,56950
1,зеленая пластинка,61,610,-920,-310
2,автограф Стаса Барецкого,48,28800,-820,27980
3,плюмбус,65,16250,-940,15310
4,статуэтка Ленина,68,13600,-935,12665
5,подписка на suppi-блог,33,4950,-545,4405
6,ломтик июльского неба,42,18900,-670,18230


**3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов**

---




In [39]:
orders_profit = pd.DataFrame(columns = ['order_id', 'order_profit'])
order_id = []
order_profit = []

# В цикле обходим все заказы и заполняем два списка: order_id и order_profit.
for i in range(len(data)):
    income_from_the_order = 0
    expenses_from_the_order = 0
    highway_cost = data[i]['highway_cost']
    order_id.append(data[i]['order_id'])

    for product in data[i]['products']:
        income_from_the_order += product['price'] * product['quantity']
    # Записываем profit как разность доходов и
    # расходов с каждого заказа (highway_cost отрицателен)
    order_profit.append(income_from_the_order + highway_cost)


In [40]:
# Проверим, совпадают ли длины списков.
assert len(order_id) == len(order_profit)

# Заполним наш DataFrame orders_profit.
orders_profit['order_id'] = order_id
orders_profit['order_profit'] = order_profit

In [41]:
# Вычислим среднюю прибыль заказов.
# Выведем её на экран вместе с первыми 10 строками таблицы order_profit.
mean_order_profit = orders_profit['order_profit'].mean()

print(f'mean order profit: {mean_order_profit}')
orders_profit.head(10)

mean order profit: 1352.3


,order_id,order_profit
0,11973,3980
1,62239,985
2,85794,-30
3,33684,1980
4,25824,975
5,87044,735
6,58598,1590
7,5430,1120
8,60502,1725
9,96473,0


**4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)**

**5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться
'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.**

**6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A.
Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'**

---



In [42]:
# Создадим функцию для определения категории
# на основании значения накопленного процента
def category_fun(row):
    value = float(row['accumulated_percent_profit_product_of_warehouse'])
    if value <= 70.0:
        category = 'A'
    elif (value > 70.0) & (value <= 90.0):
        category = 'B'
    else:
        category = 'C'
    return category

In [43]:
columns = ['warehouse_name',
            'product',
            'quantity',
            'profit'
]
warehouses = list(delivery_cost_rate_df['warehouse_name'])

# Создадим DataFrame, содержащий первый 4 стобца будущей таблицы
warehouses_df = pd.DataFrame(columns = columns)

In [51]:
# Для каждого склада будем заполнять свой DataFrame df_for_each_warehouse,
# а затем присоединять его к warehouses_df.
for warehouse in warehouses:
    total_warehouse_profit = 0
    df_for_each_warehouse = pd.DataFrame(columns = columns)
    for i in range(len(data)):
        if data[i]['warehouse_name'] == warehouse:
            for product in data[i]['products']:

                # Инициализируем основные переменные для каждого продукта
                product_name = product['product']
                quantity = product['quantity']
                income = product['price'] * quantity
                expenses = get_delivery_cost_rate(warehouse) * quantity
                profit = income + expenses
                total_warehouse_profit += profit

                # Проверка: если в DataFrame для нашего склада уже есть такой
                # продукт, то будем прибавлять количество купленного продукта
                # и прибыль с него к уже существующим значениям.
                bool_column = list(
                        df_for_each_warehouse['product'].isin([product_name]))
                if True in bool_column:
                    index = df_for_each_warehouse.index[
                               df_for_each_warehouse['product'] == product_name]

                    df_for_each_warehouse.loc[index, 'quantity'] += quantity
                    df_for_each_warehouse.loc[index, 'profit'] += profit

                # Если такого продукта в DataFrame для нашего склада ещё нет,
                # то запишем новую строку.
                else:
                    new_row = [warehouse, product_name, quantity, profit]
                    df_for_each_warehouse.loc[
                        len(df_for_each_warehouse.index)] = new_row

    # Добавим новый столбец 'percent_profit_product_of_warehouse'.
    df_for_each_warehouse['percent_profit_product_of_warehouse'
        ] = (df_for_each_warehouse['profit'] / total_warehouse_profit) * 100

    # Отсортируем DataFrame по столбцу 'percent_profit_product_of_warehouse'.
    df_for_each_warehouse = df_for_each_warehouse.sort_values(
                    by=['percent_profit_product_of_warehouse'],
                    ascending=False)

    # Добавим новый столбец 'accumulated_percent_profit_product_of_warehouse'.
    df_for_each_warehouse['accumulated_percent_profit_product_of_warehouse'
                ] = df_for_each_warehouse[
                            'percent_profit_product_of_warehouse'].cumsum()

    # Добавим новый столбец 'category'.
    df_for_each_warehouse['category'] = df_for_each_warehouse.apply(
                                                category_fun,
                                                axis = 1)

    # Присоединим DataFrame для склада df_for_each_warehouse к warehouses_df
    #warehouses_df = warehouses_df.append(df_for_each_warehouse)
    pd.concat([warehouses_df, df_for_each_warehouse])

display(warehouses_df)

,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
6,хутор близ Диканьки,автограф Стаса Барецкого,22,12870,34.559613,34.559613,A
0,хутор близ Диканьки,билет в Израиль,10,9850,26.450054,61.009667,A
1,хутор близ Диканьки,плюмбус,27,6345,17.038131,78.047798,B
4,хутор близ Диканьки,статуэтка Ленина,21,3885,10.432331,88.480129,B
5,хутор близ Диканьки,ломтик июльского неба,7,3045,8.176692,96.656821,C
...,...,...,...,...,...,...,...
1,остров невезения,статуэтка Ленина,18,3510,22.485586,50.992953,A
0,остров невезения,плюмбус,14,3430,21.973094,72.966047,B
4,остров невезения,билет в Израиль,3,2985,19.122357,92.088405,C
3,остров невезения,автограф Стаса Барецкого,2,1190,7.623318,99.711723,C
